# EO4SD SHORELINE CHANGE MAPPING AND FORECASTING

This code has been modifed by Carpenter (2020) for the project Earth Observation for Sustainable Development. Below demonstrates an example processing workflow for Benin and Togo's Coastline between 2000-2020.

This software is based on scripts and code developed by:
* Vos K., Splinter K.D., Harley M.D., Simmons J.A., Turner I.L. (2019). CoastSat: a Google Earth Engine-enabled Python toolkit to extract shorelines from publicly available satellite imagery. Environmental Modelling and Software. 122, 104528. https://doi.org/10.1016/j.envsoft.2019.104528

It enables the users to extract time-series of shoreline change over the last 20+ years at their site of interest.
There are three main steps:
1. Retrieval of median composite satellite images of the region of interest from Google Earth Engine
2. Shoreline extraction at sub-pixel resolution

## Initial settings

Refer to the Set-up and Installation section of the User Handbook for instructions on how to install the Python packages necessary to run the software, including Google Earth Engine Python API. See original methodology via https://github.com/kvos/CoastSat

In [ ]:
import os
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from utils.print_utils import printProgress, printWarning, printSuccess
from coastsat import NOC_tools, NOC_shoreline, NOC_download, SDS_tools


# directory where the data will be accessed and stored
data_partition = 'c:\\data\\coastsat'
country = 'UK'
base_dir_path = os.path.join(data_partition, country)  
sites_dir_path = os.path.join(base_dir_path, 'sites')
results_dir_path = os.path.join(base_dir_path, 'results')
sites = os.listdir(sites_dir_path)

settings = {
            'output_epsg': 27700,   
            'min_length_shoreline': 500, 
            
            'sat_name': 'S1',
            'pixel_size': 10,
            'results_dir_path': results_dir_path
            }
dates = [
         ['2016-01-01', '2016-06-30'],
         ['2016-07-01', '2016-12-31'],
         ['2017-01-01', '2017-06-30'],
         ['2017-07-01', '2017-12-31'],
         ['2018-01-01', '2018-06-30'],
         ['2018-07-01', '2018-12-31'],
         ['2019-01-01', '2019-06-30'],
         ['2019-07-01', '2019-12-31'],
         ['2020-01-01', '2020-06-30'],
         ['2020-07-01', '2020-12-31'],
                                     ]

settings['date_range'] = ['2016-01-01', '2020-12-31']

## download SAR data from GEE

In [ ]:
for site in sites:
    
    kml_filepath = os.path.join(sites_dir_path, site)
    kml_polygon = NOC_tools.polygon_from_kml(kml_filepath)
    roi_polygon = SDS_tools.smallest_rectangle(kml_polygon)

    site_name = site[:site.find('.')]
    median_dir_path = os.path.join(base_dir_path, site_name, 'median')
    settings['polygon'] = roi_polygon
    settings['site_name'] = site_name
    settings['median_dir_path'] = median_dir_path
        
    for date_pair in dates:

        settings['dates'] = date_pair 
        
        printProgress(f'processing {site_name}: {date_pair}')
        
        NOC_download.retrieve_median_sar(settings)
        
    NOC_download.save_metadata(settings)
            
print('FINISHED ...')

## find reference shoreline

In [ ]:
settings['polarisation'] = 'VH'
settings['sigma'] = 4
    
for site in sites:
    
    site_name = site[:site.find('.')]
    median_dir_path = os.path.join(base_dir_path, site_name, 'median')
    settings['site_name'] = site_name
    settings['median_dir_path'] = median_dir_path   
    
    printProgress(f'processing {site_name}')

    %matplotlib qt
    NOC_shoreline.find_reference_shoreline(settings)
            
print('FINISHED ...')

## extract shorelines

In [ ]:
settings['reference_threshold'] = -24.0
settings['polarisation'] = 'VH'
settings['max_dist_ref'] = 150

for site in sites:
    
    site_name = site[:site.find('.')]
    median_dir_path = os.path.join(base_dir_path, site_name, 'median')
    settings['site_name'] = site_name
    settings['median_dir_path'] = median_dir_path 
    NOC_shoreline.load_reference_shoreline(settings)
        
    for date_pair in dates:

        settings['dates'] = date_pair 
        
        printProgress(f'processing {site_name}: {date_pair}')

        %matplotlib qt
        NOC_shoreline.extract_shoreline_sar(settings, batch=False)
        
print('FINISHED ...')